In [9]:
import pandas as pd
from factor_analyzer import FactorAnalyzer
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import numpy as np
from statistics import median

train_df = pd.read_csv(r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\updated_features\csv_for_rulefit\train_pKa_v3.csv', index_col=0)
test_df = pd.read_csv(r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\updated_features\csv_for_rulefit\test_pKa_v3.csv', index_col=0)

merged_dataframe = pd.concat([train_df, test_df], axis=0)

merged_dataframe.rename(columns={"amine_type": "identificator"}, inplace=True)
merged_dataframe.drop(['molecule_type'], axis=1, inplace=True)

features_to_analyse = []
for feature_name in merged_dataframe.columns:
    if len(merged_dataframe[feature_name].unique()) > 1:
        features_to_analyse.append(feature_name)

features_to_drop = ['nFARing', 'nFAHRing', 'nFHRing', 'nAHRing', 'fold_id']

for feature_to_drop in features_to_drop:
    features_to_analyse.remove(feature_to_drop)

fa = FactorAnalyzer(rotation=None)

df_features = merged_dataframe[features_to_analyse]
df_features_darray = StandardScaler().fit_transform(df_features)
fa.fit(df_features_darray)

ev, _ = fa.get_eigenvalues()
for index, en_value in enumerate(ev):
    if en_value < 1:
        break
index += 1

n_factors = 6

fa = FactorAnalyzer(n_factors=n_factors, rotation="varimax")

fa.fit(df_features)

fa_load = pd.DataFrame(fa.loadings_,index=df_features.columns)
fa_load.style.background_gradient(cmap="coolwarm")

,0,1,2,3,4,5
avg_atoms_in_cycle,0.521532,0.128317,0.052926,-0.188802,0.471597,-0.190548
chirality,0.087506,-0.133943,0.139979,-0.049615,-0.103545,0.753437
PPSA5,0.092767,-0.853650,0.025490,0.342873,0.125320,-0.013395
RPCS,-0.201265,0.281419,-0.000074,-0.624055,-0.059486,0.043598
mol_num_cycles,0.092932,0.040879,0.942522,-0.088552,0.096134,0.014319
GeomShapeIndex,-0.075512,-0.524651,-0.025768,0.195480,-0.002915,0.087949
angle_R2X2R1,0.955434,-0.044130,-0.009687,0.193672,0.076656,0.065492
nN,0.059500,0.966468,-0.032068,0.028867,-0.122038,-0.017250
distance_between_atoms_in_f_group_centers,0.752809,0.162035,0.080584,0.203489,0.060982,0.007549
nC,0.127049,-0.211892,0.595066,0.045006,0.751276,-0.079384


In [17]:
medians_for_features = {}
for feature_name in df_features.columns:
    medians_for_features[feature_name] = median(df_features[feature_name])

medians_for_features

{'avg_atoms_in_cycle': 4.0,
 'chirality': 0,
 'PPSA5': 11.306940529954163,
 'RPCS': 16.5901927054261,
 'mol_num_cycles': 1,
 'GeomShapeIndex': 0.6422838373782467,
 'angle_R2X2R1': 103.44173207136691,
 'nN': 1,
 'distance_between_atoms_in_f_group_centers': 1.498352188301075,
 'nC': 5,
 'angle_R1X1R2': 99.26138155009872,
 'f_freedom': 1,
 'cis/trans': 0,
 'dipole_moment': 0.3913210742586738,
 'f_to_fg': 2.0,
 'nFRing': 0,
 'PBF': 0.670246346273359,
 'nARing': 1,
 'nF': 2,
 'dihedral_angle': 2.6122304011075688,
 'nO': 0,
 'TASA': 191.30629717730744,
 'angle_X2X1R1': 111.73009521109194,
 'mol_volume': 110.85,
 'FPSA3': 0.0373397997635106,
 'PNSA5': -16.940435357676453,
 'angle_X1X2R2': 116.07898596286806,
 'nHRing': 0,
 'identificator': 1,
 'pKa': 7.78}

In [32]:
features_names = df_features.columns
fa_loadings = fa.loadings_

for factor_index in range(n_factors):
    important_features_in_factor = []
    is_inverted = []
    for feature_index in range(len(fa_loadings[:,factor_index])):
        if abs(fa_loadings[feature_index][factor_index]) > 0.7 and features_names[feature_index] != 'pKa':
            important_features_in_factor.append(features_names[feature_index])
            if fa_loadings[feature_index][factor_index] > 0: is_inverted.append(False)
            else: is_inverted.append(True)
    
    print(important_features_in_factor)
    print(is_inverted)
    
    
    lower_pKa = set()
    higher_pKa = set()
    for important_feature_in_factor_index in range(len(important_features_in_factor)):
        feature_name = important_features_in_factor[important_feature_in_factor_index]
        for index, row in df_features.iterrows():
            if row[feature_name] > medians_for_features[feature_name] and is_inverted[important_feature_in_factor_index] == False or \
                row[feature_name] < medians_for_features[feature_name] and is_inverted[important_feature_in_factor_index] == True:        
                higher_pKa.add(row['pKa'])
            if row[feature_name] >= medians_for_features[feature_name] and is_inverted[important_feature_in_factor_index] == True or \
                row[feature_name] <= medians_for_features[feature_name] and is_inverted[important_feature_in_factor_index] == False:        
                lower_pKa.add(row['pKa'])
    
    print(f"higher_pKa: {sum(higher_pKa) / len(higher_pKa)}")
    print(f"lower_pKa: {sum(lower_pKa) / len(lower_pKa)}")

    print("-"*20)


['angle_R2X2R1', 'distance_between_atoms_in_f_group_centers', 'angle_R1X1R2', 'f_to_fg', 'angle_X2X1R1', 'angle_X1X2R2']
[False, False, False, False, False, False]
higher_pKa: 7.096304347826084
lower_pKa: 7.5412056737588635
--------------------
['PPSA5', 'nN', 'nO', 'FPSA3', 'PNSA5', 'identificator']
[True, False, True, True, False, False]
higher_pKa: 8.449729729729727
lower_pKa: 7.517945205479451
--------------------
['mol_num_cycles', 'nFRing', 'nARing']
[False, False, False]
higher_pKa: 7.788857142857145
lower_pKa: 7.369999999999997
--------------------
['f_freedom', 'nF']
[True, False]
higher_pKa: 6.5356250000000005
lower_pKa: 7.655163934426227
--------------------
['nC', 'mol_volume']
[False, False]
higher_pKa: 6.955949367088609
lower_pKa: 7.956111111111108
--------------------
['chirality', 'cis/trans']
[False, False]
higher_pKa: 6.671463414634147
lower_pKa: 7.598347107438013
--------------------


Only for amine molecules

In [34]:
import os
import sys
sys.path.insert(0, os.path.dirname('C:\work\DrugDiscovery\main_git\XAI_Chem\ml_part'))

import pandas as pd

from ml_part.random_forest.data_prep.preparation import DataPreparation
from ml_part.random_forest.train import RFTrain

CSV_PATH = r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\updated_features\remained_features_pKa_01.02_v2.csv'
smiles_filepath = r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\updated_features\smiles_to_index.pkl'

dataPreparation = DataPreparation(CSV_PATH)

unimportant_features_to_drop = ['logP']
X, y = dataPreparation.prepare_data_for_RF(is_pKa=True,
                                           molecule_type="amine",
                                           use_mandatory_features=True,
                                           is_remove_outliers=True,
                                           is_remove_nan=False,
                                           outliers_features_to_skip=unimportant_features_to_drop,)

correlated_features = ['f_atom_fraction', 'naHRing', 'nFaRing', 'nFaHRing', 'tpsa+f']
features_to_drop = []
for feature_name in X.columns:
    if feature_name in correlated_features:
        features_to_drop.append(feature_name)

X = X.drop(features_to_drop, axis=1)

rf_train = RFTrain(X=X, 
                   y=y,
                   smiles_filepath=smiles_filepath,
                   is_pKa=True,
                   k_folds=2)

y_train = rf_train.y_train
X_train = rf_train.X_train

y_test = rf_train.y_test
X_test = rf_train.X_test

train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

print(len(train_df), len(test_df))

True
128
['avg_atoms_in_cycle', 'nFaHRing', 'chirality', 'PPSA5', 'tpsa+f', 'RPCS', 'mol_num_cycles', 'GeomShapeIndex', 'angle_R2X2R1', 'nN', 'distance_between_atoms_in_f_group_centers', 'nC', 'nFARing', 'angle_R1X1R2', 'f_freedom', 'naHRing', 'nFAHRing', 'cis/trans', 'dipole_moment', 'f_to_fg', 'identificator', 'f_atom_fraction', 'nFRing', 'nFaRing', 'naRing', 'nFHRing', 'PBF', 'nARing', 'nF', 'dihedral_angle', 'nAHRing', 'nO', 'TASA', 'angle_X2X1R1', 'mol_volume', 'FPSA3', 'PNSA5', 'angle_X1X2R2', 'nHRing', 'pKa', 'logP']
dipole_moment outliers indexes: [27]
f_atom_fraction outliers indexes: [69]
mol_volume outliers indexes: [72]
Remains rows:121, amount of features: 41
102 19


In [161]:
from factor_analyzer import FactorAnalyzer
from sklearn.preprocessing import StandardScaler
import pandas as pd
from factor_analyzer import FactorAnalyzer
import matplotlib.pyplot as plt
import seaborn as sns

merged_dataframe = pd.concat([train_df, test_df], axis=0)

# merged_dataframe.rename(columns={"amine_type": "identificator"}, inplace=True)

# indexes_to_drop = []
# for index, row in merged_dataframe.iterrows():
#     if row['distance_between_atoms_in_f_group_centers'] == 0:
#         indexes_to_drop.append(index)
# merged_dataframe.drop(indexes_to_drop, axis=0, inplace=True)

features_to_analyse = []
for feature_name in merged_dataframe.columns:
    if len(merged_dataframe[feature_name].unique()) > 1:
        features_to_analyse.append(feature_name)

features_to_drop = ['nFARing', 'nFAHRing', 'nFHRing', 'nAHRing', 'fold_id']

for feature_to_drop in features_to_drop:
    features_to_analyse.remove(feature_to_drop)

fa = FactorAnalyzer(rotation=None)


df_features = merged_dataframe[features_to_analyse]
df_features_darray = StandardScaler().fit_transform(df_features)
fa.fit(df_features_darray)

ev, _ = fa.get_eigenvalues()
for index, en_value in enumerate(ev):
    if en_value < 1:
        break
index += 1

n_factors = 7

fa = FactorAnalyzer(n_factors=n_factors, rotation="varimax")
fa.fit(df_features)

fa_load = pd.DataFrame(fa.loadings_,index=df_features.columns)
fa_load.style.background_gradient(cmap="coolwarm")

,0,1,2,3,4,5,6
avg_atoms_in_cycle,0.559514,-0.128207,0.009668,0.313833,-0.016495,0.492427,-0.194931
chirality,0.069613,0.029592,0.043625,-0.074205,0.880531,-0.058600,0.020370
PPSA5,0.181451,0.873914,-0.037794,-0.017925,-0.003019,0.270724,0.094578
RPCS,-0.258680,-0.692308,-0.044495,0.122418,0.146268,0.096657,0.188542
mol_num_cycles,0.100730,-0.138818,0.895782,0.218132,0.068037,-0.014289,-0.141784
GeomShapeIndex,-0.090977,0.280549,-0.059909,-0.342687,-0.020933,-0.177356,0.001055
angle_R2X2R1,0.945721,0.304170,0.024254,0.004577,0.054207,0.050896,0.008200
distance_between_atoms_in_f_group_centers,0.726726,0.281742,0.093868,0.113450,0.022852,0.044012,-0.044925
nC,0.168151,0.015929,0.766467,0.041017,-0.102299,0.594800,-0.120563
angle_R1X1R2,0.928906,0.308957,0.003559,0.147516,0.088919,-0.002026,0.020943


In [46]:
medians_for_features = {}
for feature_name in df_features.columns:
    medians_for_features[feature_name] = median(df_features[feature_name])

medians_for_features

{'avg_atoms_in_cycle': 4.0,
 'chirality': 0,
 'PPSA5': 10.452837258190042,
 'RPCS': 7.150318535820773,
 'mol_num_cycles': 1,
 'GeomShapeIndex': 0.5851234407622666,
 'angle_R2X2R1': 100.17253184454503,
 'distance_between_atoms_in_f_group_centers': 1.5285813872197025,
 'nC': 5,
 'angle_R1X1R2': 99.6408048541743,
 'f_freedom': 1,
 'cis/trans': 0,
 'dipole_moment': 0.3931144784664414,
 'f_to_fg': 2.0,
 'identificator': 2,
 'nFRing': 0,
 'PBF': 0.6589667081744836,
 'nARing': 1,
 'nF': 2,
 'dihedral_angle': 2.996612139414393,
 'nO': 0,
 'TASA': 211.36519837472065,
 'angle_X2X1R1': 115.17801686108022,
 'mol_volume': 106.57,
 'FPSA3': 0.034808443256566,
 'PNSA5': -14.710723928759617,
 'angle_X1X2R2': 115.92053903765051,
 'nHRing': 1,
 'pKa': 9.05}

In [49]:
features_names = df_features.columns
fa_loadings = fa.loadings_

for factor_index in range(n_factors):
    important_features_in_factor = []
    is_inverted = []
    for feature_index in range(len(fa_loadings[:,factor_index])):
        if abs(fa_loadings[feature_index][factor_index]) > 0.5 and features_names[feature_index] != 'pKa':
            important_features_in_factor.append(features_names[feature_index])
            if fa_loadings[feature_index][factor_index] > 0: is_inverted.append(False)
            else: is_inverted.append(True)
    
    print(f"Factor number: {factor_index}")
    print(important_features_in_factor)
    print(is_inverted)
    
    
    lower_pKa = set()
    higher_pKa = set()
    for important_feature_in_factor_index in range(len(important_features_in_factor)):
        feature_name = important_features_in_factor[important_feature_in_factor_index]
        for index, row in df_features.iterrows():
            if row[feature_name] > medians_for_features[feature_name] and is_inverted[important_feature_in_factor_index] == False or \
                row[feature_name] < medians_for_features[feature_name] and is_inverted[important_feature_in_factor_index] == True:        
                higher_pKa.add(row['pKa'])
            if row[feature_name] >= medians_for_features[feature_name] and is_inverted[important_feature_in_factor_index] == True or \
                row[feature_name] <= medians_for_features[feature_name] and is_inverted[important_feature_in_factor_index] == False:        
                lower_pKa.add(row['pKa'])
    
    print(f"higher_pKa: {sum(higher_pKa) / len(higher_pKa)}")
    print(f"lower_pKa: {sum(lower_pKa) / len(lower_pKa)}")

    print("-"*20)


Factor number: 0
['avg_atoms_in_cycle', 'angle_R2X2R1', 'distance_between_atoms_in_f_group_centers', 'angle_R1X1R2', 'f_to_fg', 'dihedral_angle', 'angle_X2X1R1', 'angle_X1X2R2']
[False, False, False, False, False, False, False, False]
higher_pKa: 8.480126582278478
lower_pKa: 8.892621359223297
--------------------
Factor number: 1
['PPSA5', 'RPCS', 'f_freedom', 'dipole_moment', 'f_to_fg', 'nF', 'PNSA5']
[False, True, True, False, False, False, True]
higher_pKa: 8.10756756756757
lower_pKa: 8.844339622641504
--------------------
Factor number: 2
['mol_num_cycles', 'nC', 'nFRing', 'PBF', 'nARing', 'mol_volume']
[False, False, False, False, False, False]
higher_pKa: 8.688533333333336
lower_pKa: 8.869230769230766
--------------------
Factor number: 3
['identificator', 'nHRing']
[False, False]
higher_pKa: 9.052222222222222
lower_pKa: 8.798611111111109
--------------------
Factor number: 4
['chirality', 'cis/trans']
[False, False]
higher_pKa: 8.2816
lower_pKa: 8.964269662921344
---------------

compare chirality/cis-trans

In [122]:
important_features_in_factor = ['chirality', 'cis/trans']
is_inverted = [False, False]

lower_pKa = []
higher_pKa = []
for index, row in df_features.iterrows():
    if row['cis/trans'] == 0 and row['chirality'] == 0:
        higher_pKa.append(row['pKa'])
    if row['cis/trans'] == 1 and row['chirality'] != 0:
        lower_pKa.append(row['pKa'])
    # if row['chirality'] == 0:
    #     print(row['chirality'], row['pKa'])
    #     higher_pKa.append(row['pKa'])
    # if row['chirality'] > 0:
    #     lower_pKa.append(row['pKa'])
    #     print(row['chirality'], row['pKa'])

print(len(higher_pKa))
print(len(lower_pKa))

if len(higher_pKa) > 0: print(f"higher_pKa: {sum(higher_pKa) / len(higher_pKa)}")
if len(lower_pKa) > 0: print(f"lower_pKa: {sum(lower_pKa) / len(lower_pKa)}")

96
8
higher_pKa: 8.988020833333339
lower_pKa: 7.84875


In [132]:
median_angle_value = median(df_features[df_features['angle_X1X2R2'] > 0]['angle_X1X2R2'])
median_angle_value

136.66370418803444

Angles investigation

top angle in features: R1X1R2

In [133]:
important_features_in_factor = ['chirality', 'cis/trans']
is_inverted = [False, False]

lower_pKa = []
higher_pKa = []
for index, row in df_features.iterrows():
    if row['angle_X1X2R2'] == 0: continue
    if row['angle_X1X2R2'] > median_angle_value:
        higher_pKa.append(row['pKa'])
    if row['angle_X1X2R2'] < median_angle_value:
        lower_pKa.append(row['pKa'])
    # print(row['angle_R1X1R2'], row['pKa'])

print(len(higher_pKa))
print(len(lower_pKa))

print(f"higher_pKa: {sum(higher_pKa) / len(higher_pKa)}")
print(f"lower_pKa: {sum(lower_pKa) / len(lower_pKa)}")

38
38
higher_pKa: 8.487894736842106
lower_pKa: 7.895526315789475


is functional groups in one side:

In [128]:
lower_pKa = []
higher_pKa = []
for index, row in df_features.iterrows():
    if row['angle_R1X1R2'] == 0 or row['angle_X2X1R1'] == 0: continue
    if row['angle_R1X1R2'] > row['angle_X2X1R1']: # different sides
        # print("trans", row['cis/trans'])
        higher_pKa.append(row['pKa'])
    if row['angle_R1X1R2'] < row['angle_X2X1R1']: # same sides
        # print("cis", row['cis/trans'])
        lower_pKa.append(row['pKa'])
    # print(row['angle_R1X1R2'], row['pKa'])

print(len(higher_pKa))
print(len(lower_pKa))

print(f"higher_pKa: {sum(higher_pKa) / len(higher_pKa)}")
print(f"lower_pKa: {sum(lower_pKa) / len(lower_pKa)}")

32
44
higher_pKa: 8.4140625
lower_pKa: 8.030000000000003


short and long distances

In [157]:
median_distance_value = median(df_features[df_features['distance_between_atoms_in_f_group_centers'] > 0]['distance_between_atoms_in_f_group_centers'])
print("median:", median_distance_value)

lower_pKa = []
higher_pKa = []
for index, row in df_features.iterrows():
    if row['distance_between_atoms_in_f_group_centers'] == 0: continue
    if row['distance_between_atoms_in_f_group_centers'] > median_distance_value + 1: # different sides
        # print("trans", row['cis/trans'])
        higher_pKa.append(row['pKa'])
    if row['distance_between_atoms_in_f_group_centers'] < median_distance_value - 1: # same sides
        # print("cis", row['cis/trans'])
        lower_pKa.append(row['pKa'])
    # print(row['angle_R1X1R2'], row['pKa'])

print(len(higher_pKa))
print(len(lower_pKa))

print(f"higher_pKa: {sum(higher_pKa) / len(higher_pKa)}")
print(f"lower_pKa: {sum(lower_pKa) / len(lower_pKa)}")

median: 2.944062277480133
20
21
higher_pKa: 8.796000000000001
lower_pKa: 8.345238095238095


identificator check

In [152]:
lower_pKa = []
higher_pKa = []
for index, row in df_features.iterrows():
    if row['identificator'] == 1: # different sides
        # print("trans", row['cis/trans'])
        higher_pKa.append(row['pKa'])
    if row['identificator'] == 2: # same sides
        # print("cis", row['cis/trans'])
        lower_pKa.append(row['pKa'])
    # print(row['angle_R1X1R2'], row['pKa'])

print(f"higher_pKa: {sum(higher_pKa) / len(higher_pKa)}")
print(f"lower_pKa: {sum(lower_pKa) / len(lower_pKa)}")

higher_pKa: 8.58655172413793
lower_pKa: 7.948085106382981


dipole_moment

In [171]:
median_value = median(df_features['PPSA5'])
print("median:", median_value)

lower_pKa = []
higher_pKa = []
for index, row in df_features.iterrows():
    if row['PPSA5'] == 0: continue
    if row['PPSA5'] > median_value: # different sides
        # print("trans", row['cis/trans'])
        higher_pKa.append(row['pKa'])
    if row['PPSA5'] < median_value: # same sides
        # print("cis", row['cis/trans'])
        lower_pKa.append(row['pKa'])
    # print(row['angle_R1X1R2'], row['pKa'])

print(len(higher_pKa))
print(len(lower_pKa))

print(f"higher_pKa: {sum(higher_pKa) / len(higher_pKa)}")
print(f"lower_pKa: {sum(lower_pKa) / len(lower_pKa)}")

median: 10.452837258190042
60
60
higher_pKa: 7.977000000000003
lower_pKa: 9.697000000000001
